# MedHacks - SaRS Slayers

### Import data

In [2]:
import pandas as pd
import numpy as np
import pandas as pd

Reading data:

In [10]:
covid19_dataframe = pd.read_csv('datasets/covid-chestxray-dataset-master/metadata.csv')

Looking at the csv file:

In [11]:
covid19_dataframe.sample(5)

,patientid,offset,sex,age,finding,RT_PCR_positive,survival,intubated,intubation_present,went_icu,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 29
545,290,0.0,F,63.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,Y,...,2020,"Brescia, Italy",images,270_2020_2470_Fig1_HTML-a.png,10.1007/s00270-020-02470-0,https://link.springer.com/article/10.1007%2Fs0...,NaN,A 63-year-old woman with metastatic ovarian ca...,NaN,NaN
63,29,5.0,NaN,65.0,Pneumonia/Bacterial/Streptococcus,NaN,NaN,NaN,NaN,NaN,...,"May 9, 2019","Laniado Hospital, Netanya, Israel",images,streptococcus-pneumoniae-pneumonia-1.jpg,NaN,https://radiopaedia.org/cases/streptococcus-pn...,CC BY-NC-SA,"Large consolidations in the right upper lobe, ...","Case courtesy of Dr Yair Glick, Radiopaedia.or...",NaN
476,252,27.0,F,NaN,Pneumonia/Viral/COVID-19,Unclear,NaN,NaN,NaN,N,...,2020,"Hannover Medical School, Hannover, Germany",images,c601f50d.jpg,10.6084/m9.figshare.12275009,https://github.com/ml-workgroup/covid-19-image...,CC BY 3.0,NaN,NaN,NaN
155,84,NaN,M,60.0,Pneumonia/Viral/COVID-19,Unclear,NaN,NaN,NaN,NaN,...,"Mar 3, 2020",Italy,images,2B8649B2-00C4-4233-85D5-1CE240CF233B.jpeg,NaN,https://www.sirm.org/2020/03/21/covid-19-caso-58/,NaN,60 year old man Dyspnea and hyperthermia appe...,"Credit to P. Nespoli, P. Moretto, D. Sardo, P....",NaN
878,440,0.0,M,NaN,todo,NaN,NaN,NaN,NaN,N,...,2020,"Hannover Medical School, Hannover, Germany",images,1d6c4b7c.jpg,10.6084/m9.figshare.12275009,https://github.com/ml-workgroup/covid-19-image...,CC BY 3.0,NaN,NaN,NaN


In [12]:
covid19_dataframe.shape

(950, 30)

Looking into these categories:

In [13]:
covid19_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   patientid               950 non-null    object 
 1   offset                  697 non-null    float64
 2   sex                     870 non-null    object 
 3   age                     713 non-null    float64
 4   finding                 950 non-null    object 
 5   RT_PCR_positive         593 non-null    object 
 6   survival                361 non-null    object 
 7   intubated               248 non-null    object 
 8   intubation_present      250 non-null    object 
 9   went_icu                397 non-null    object 
 10  in_icu                  335 non-null    object 
 11  needed_supplemental_O2  90 non-null     object 
 12  extubated               37 non-null     object 
 13  temperature             78 non-null     float64
 14  pO2_saturation          119 non-null    fl

Lets look at findings, so we can target Pneumonia

In [15]:
covid19_dataframe['finding'].unique()

array(['Pneumonia/Viral/COVID-19', 'Pneumonia', 'Pneumonia/Viral/SARS',
       'Pneumonia/Fungal/Pneumocystis',
       'Pneumonia/Bacterial/Streptococcus', 'No Finding',
       'Pneumonia/Bacterial/Chlamydophila', 'Pneumonia/Bacterial/E.Coli',
       'Pneumonia/Bacterial/Klebsiella', 'Pneumonia/Bacterial/Legionella',
       'Unknown', 'Pneumonia/Lipoid', 'Pneumonia/Viral/Varicella',
       'Pneumonia/Bacterial', 'Pneumonia/Bacterial/Mycoplasma',
       'Pneumonia/Viral/Influenza', 'todo', 'Tuberculosis',
       'Pneumonia/Viral/Influenza/H1N1', 'Pneumonia/Fungal/Aspergillosis',
       'Pneumonia/Viral/Herpes ', 'Pneumonia/Aspiration',
       'Pneumonia/Bacterial/Nocardia', 'Pneumonia/Viral/MERS-CoV',
       'Pneumonia/Bacterial/Staphylococcus/MRSA'], dtype=object)

So interesting , how many different findings are here.

In [16]:
covid19_dataframe['view'].unique()

array(['PA', 'AP', 'L', 'Axial', 'AP Supine', 'Coronal', 'AP Erect'],
      dtype=object)

So, based on some google search:

- PA - Posteroanterior View (Frontal View of Chest X-rays)
- AP - Anteroposterior View ( front-to-back)

- ....and so forth.

I think for simplicity, I will only be Frontal view of Chest X-rays

Lets target COVID-19 in findings and PA (frontal view) in view column.

In [23]:
COVID_PA_Xray = covid19_dataframe.loc[(covid19_dataframe['finding'] == 'Pneumonia/Viral/COVID-19') & (covid19_dataframe['view'] == 'PA')]
COVID_PA_Xray

,patientid,offset,sex,age,finding,RT_PCR_positive,survival,intubated,intubation_present,went_icu,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 29
0,2,0.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
1,2,3.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 25, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
2,2,5.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 27, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
3,2,6.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 28, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
4,4,0.0,F,52.0,Pneumonia/Viral/COVID-19,Y,NaN,N,N,N,...,"January 25, 2020","Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,471,30.0,M,20.0,Pneumonia/Viral/COVID-19,Y,NaN,NaN,NaN,NaN,...,2020,United Kingdom,images,16953_3_1.jpg,NaN,https://www.eurorad.org/case/16953,CC BY-NC-SA 4.0,A 20-year-old male with a history of asthma pr...,NaN,NaN
932,471,51.0,M,20.0,Pneumonia/Viral/COVID-19,Y,NaN,NaN,NaN,NaN,...,2020,United Kingdom,images,16953_4_1.jpg,NaN,https://www.eurorad.org/case/16953,CC BY-NC-SA 4.0,A 20-year-old male with a history of asthma pr...,NaN,NaN
933,471,72.0,M,20.0,Pneumonia/Viral/COVID-19,Y,NaN,NaN,NaN,NaN,...,2020,United Kingdom,images,16953_5_1.jpg,NaN,https://www.eurorad.org/case/16953,CC BY-NC-SA 4.0,A 20-year-old male with a history of asthma pr...,NaN,NaN
936,474,NaN,M,55.0,Pneumonia/Viral/COVID-19,Unclear,NaN,NaN,NaN,NaN,...,2020,"Al Hasa, Saudi Arabia",images,2edb88df42cab5e5fbc18b3965e0bd_jumbo.jpeg,NaN,https://radiopaedia.org/cases/covid-19-pneumon...,CC BY-NC-SA,"Fever, abdominal pain and diarrhea. No cough o...","Case courtesy of Dr Osama Rizk, Radiopaedia.o...",NaN


In [25]:
# Number of COVID-19 xrays with frontal view
COVID_PA_Xray.shape[0]

196

In [30]:
import os
import shutil

Extract images to a different folder

In [29]:
image_count = 0

for(i,row) in COVID_PA_Xray.iterrows():
    file_name=row['filename']
    image_path = os.path.join("datasets/covid-chestxray-dataset-master/images",file_name)
    copy_to_path = os.path.join("extracted_data/COVID-19_Xray",file_name)
    shutil.copy2(image_path,copy_to_path)
    image_count += 1
    print(f"{image_count} number of images moved to COVID-19_Xray Folder")

1 number of images moved to COVID-19_Xray Folder
2 number of images moved to COVID-19_Xray Folder
3 number of images moved to COVID-19_Xray Folder
4 number of images moved to COVID-19_Xray Folder
5 number of images moved to COVID-19_Xray Folder
6 number of images moved to COVID-19_Xray Folder
7 number of images moved to COVID-19_Xray Folder
8 number of images moved to COVID-19_Xray Folder
9 number of images moved to COVID-19_Xray Folder
10 number of images moved to COVID-19_Xray Folder
11 number of images moved to COVID-19_Xray Folder
12 number of images moved to COVID-19_Xray Folder
13 number of images moved to COVID-19_Xray Folder
14 number of images moved to COVID-19_Xray Folder
15 number of images moved to COVID-19_Xray Folder
16 number of images moved to COVID-19_Xray Folder
17 number of images moved to COVID-19_Xray Folder
18 number of images moved to COVID-19_Xray Folder
19 number of images moved to COVID-19_Xray Folder
20 number of images moved to COVID-19_Xray Folder
21 number

179 number of images moved to COVID-19_Xray Folder
180 number of images moved to COVID-19_Xray Folder
181 number of images moved to COVID-19_Xray Folder
182 number of images moved to COVID-19_Xray Folder
183 number of images moved to COVID-19_Xray Folder
184 number of images moved to COVID-19_Xray Folder
185 number of images moved to COVID-19_Xray Folder
186 number of images moved to COVID-19_Xray Folder
187 number of images moved to COVID-19_Xray Folder
188 number of images moved to COVID-19_Xray Folder
189 number of images moved to COVID-19_Xray Folder
190 number of images moved to COVID-19_Xray Folder
191 number of images moved to COVID-19_Xray Folder
192 number of images moved to COVID-19_Xray Folder
193 number of images moved to COVID-19_Xray Folder
194 number of images moved to COVID-19_Xray Folder
195 number of images moved to COVID-19_Xray Folder
196 number of images moved to COVID-19_Xray Folder


---

### Kaggle Dataset:

In [37]:
import random

In [47]:
normal_xray_names = os.listdir("datasets/chest_xray/train/NORMAL")

In [49]:
random.shuffle(normal_xray_names) #Shuffle to take random pics

Since we have 196 COVID-19 X-RAY pics, we will also take 196 Normal X-RAY pics in our data.

In [50]:
for i in range(196):
    image_name = normal_xray_names[i]
    shutil.copy2(os.path.join("datasets/chest_xray/train/NORMAL", image_name), os.path.join("extracted_data/Normal_Xray",image_name))
    print(f"{i} number of images moved to Normal_Xray Folder")

0 number of images moved to Normal_Xray Folder
1 number of images moved to Normal_Xray Folder
2 number of images moved to Normal_Xray Folder
3 number of images moved to Normal_Xray Folder
4 number of images moved to Normal_Xray Folder
5 number of images moved to Normal_Xray Folder
6 number of images moved to Normal_Xray Folder
7 number of images moved to Normal_Xray Folder
8 number of images moved to Normal_Xray Folder
9 number of images moved to Normal_Xray Folder
10 number of images moved to Normal_Xray Folder
11 number of images moved to Normal_Xray Folder
12 number of images moved to Normal_Xray Folder
13 number of images moved to Normal_Xray Folder
14 number of images moved to Normal_Xray Folder
15 number of images moved to Normal_Xray Folder
16 number of images moved to Normal_Xray Folder
17 number of images moved to Normal_Xray Folder
18 number of images moved to Normal_Xray Folder
19 number of images moved to Normal_Xray Folder
20 number of images moved to Normal_Xray Folder
21

171 number of images moved to Normal_Xray Folder
172 number of images moved to Normal_Xray Folder
173 number of images moved to Normal_Xray Folder
174 number of images moved to Normal_Xray Folder
175 number of images moved to Normal_Xray Folder
176 number of images moved to Normal_Xray Folder
177 number of images moved to Normal_Xray Folder
178 number of images moved to Normal_Xray Folder
179 number of images moved to Normal_Xray Folder
180 number of images moved to Normal_Xray Folder
181 number of images moved to Normal_Xray Folder
182 number of images moved to Normal_Xray Folder
183 number of images moved to Normal_Xray Folder
184 number of images moved to Normal_Xray Folder
185 number of images moved to Normal_Xray Folder
186 number of images moved to Normal_Xray Folder
187 number of images moved to Normal_Xray Folder
188 number of images moved to Normal_Xray Folder
189 number of images moved to Normal_Xray Folder
190 number of images moved to Normal_Xray Folder
191 number of images

In [51]:
pneumonia_xray_names = os.listdir("datasets/chest_xray/train/PNEUMONIA")

In [53]:
random.shuffle(pneumonia_xray_names)

In [55]:
for i in range(196):
    image_name = pneumonia_xray_names[i]
    shutil.copy2(os.path.join("datasets/chest_xray/train/PNEUMONIA", image_name), os.path.join("extracted_data/PNEUMONIA_Xray",image_name))
    print(f"{i} number of images moved to PNEUMONIA_Xray Folder")

0 number of images moved to PNEUMONIA_Xray Folder
1 number of images moved to PNEUMONIA_Xray Folder
2 number of images moved to PNEUMONIA_Xray Folder
3 number of images moved to PNEUMONIA_Xray Folder
4 number of images moved to PNEUMONIA_Xray Folder
5 number of images moved to PNEUMONIA_Xray Folder
6 number of images moved to PNEUMONIA_Xray Folder
7 number of images moved to PNEUMONIA_Xray Folder
8 number of images moved to PNEUMONIA_Xray Folder
9 number of images moved to PNEUMONIA_Xray Folder
10 number of images moved to PNEUMONIA_Xray Folder
11 number of images moved to PNEUMONIA_Xray Folder
12 number of images moved to PNEUMONIA_Xray Folder
13 number of images moved to PNEUMONIA_Xray Folder
14 number of images moved to PNEUMONIA_Xray Folder
15 number of images moved to PNEUMONIA_Xray Folder
16 number of images moved to PNEUMONIA_Xray Folder
17 number of images moved to PNEUMONIA_Xray Folder
18 number of images moved to PNEUMONIA_Xray Folder
19 number of images moved to PNEUMONIA_Xr

176 number of images moved to PNEUMONIA_Xray Folder
177 number of images moved to PNEUMONIA_Xray Folder
178 number of images moved to PNEUMONIA_Xray Folder
179 number of images moved to PNEUMONIA_Xray Folder
180 number of images moved to PNEUMONIA_Xray Folder
181 number of images moved to PNEUMONIA_Xray Folder
182 number of images moved to PNEUMONIA_Xray Folder
183 number of images moved to PNEUMONIA_Xray Folder
184 number of images moved to PNEUMONIA_Xray Folder
185 number of images moved to PNEUMONIA_Xray Folder
186 number of images moved to PNEUMONIA_Xray Folder
187 number of images moved to PNEUMONIA_Xray Folder
188 number of images moved to PNEUMONIA_Xray Folder
189 number of images moved to PNEUMONIA_Xray Folder
190 number of images moved to PNEUMONIA_Xray Folder
191 number of images moved to PNEUMONIA_Xray Folder
192 number of images moved to PNEUMONIA_Xray Folder
193 number of images moved to PNEUMONIA_Xray Folder
194 number of images moved to PNEUMONIA_Xray Folder
195 number o

Split the extracted_data folder into train, test and val datasets

In [73]:
import splitfolders
splitfolders.ratio('extracted_data', output="train-test-val_datasets", seed=123, ratio=(.8, 0.1, 0.1))

Copying files: 588 files [00:00, 646.88 files/s]
